In [ ]:
import datetime as dt

import mplfinance as mpf
import pandas as pd
import numpy as np
import pandera as pa
import matplotlib.pyplot as plt

import shared


logger = shared.get_logger()

In [ ]:
klines_btc_1m_df = shared.read_binance_klines()
klines_btc_1h_df = shared.split_df(df=klines_btc_1m_df, interval_str='1h')

In [ ]:
window_prices = []
window = 5
for i in range(len(klines_btc_1h_df) - window):
    start_row = klines_btc_1h_df.iloc[i]
    min_price = klines_btc_1h_df[i+1:i+window].low.min()
    diff_abs = start_row.high - min_price
    diff_rel = diff_abs / start_row.high * 100
    o = {'diff_abs': diff_abs, 'diff_rel': diff_rel, 'open_time': start_row.open_time}
    window_prices.append(o)
window_prices_df = pd.DataFrame(window_prices)

In [ ]:
(window_prices_df
 .assign(
     date_str=lambda x: x.open_time.apply(lambda x: dt.datetime.fromtimestamp(x/1000).date().isoformat()),
     diff_rel_int=lambda x: x.diff_rel.astype(int),
 )
 .sort_values(by=['diff_rel'], ascending=False)
 .drop_duplicates(subset=['date_str'])
 .groupby(by=['diff_rel_int'], as_index=False)
 .agg(amount=('open_time', 'count'))
 .sort_values(by=['diff_rel_int'])
)

In [ ]:
(window_prices_df
 [lambda x: x.diff_rel > 4]
 .sort_values(by=['open_time', 'diff_rel'])
 .assign(date_str=lambda x: x.open_time.apply(lambda x: dt.datetime.fromtimestamp(x/1000).date().isoformat()))
 .drop_duplicates(subset=['date_str'], keep='first')
)

In [ ]:
date_start_str, date_end_str = '2024-01-01', '2024-01-10'
slice_df = (klines_btc_1h_df
    [lambda x: x.index >= dt.datetime.fromisoformat(date_start_str)]
    [lambda x: x.index <= dt.datetime.fromisoformat(date_end_str)]
)
_, ax_list = mpf.plot(
    slice_df,
    warn_too_much_data=len(slice_df),
    type='candle',
    figsize=(14, 6),
    style=shared.s,
    volume=True,
    returnfig=True,
    xrotation=90,
    ema=(30, 60, 120),
)
ticks = pd.date_range(slice_df.index.min(), slice_df.index.max(), freq='1d')
ticks_locations = [slice_df.index.get_loc(tick) for tick in ticks]
tick_labels = [tick.strftime('%Y-%m-%d %H:%M') for tick in ticks]
_ = ax_list[-2].xaxis.set_ticks(ticks_locations)
_ = ax_list[-2].set_xticklabels(tick_labels)